# Criminal Patterns and Police Responses: An Analysis of the Montgomery County Police Crime Report

As Police departments are a publicly funded entity, we have access to a great deal of crime data. In analyzing this crime data from the Montgomery County police department, we would like to explore two main topics. First we would like to find as many patterns in the crime as possible and second, determine the safety levels of cities throughout the county.

For this analysis we will be looking at crime in [Montgomery County, Maryland](https://en.wikipedia.org/wiki/Montgomery_County,_Maryland). The data we will be working with can be found [here](https://data.montgomerycountymd.gov/Public-Safety/Crime/icn6-v9z3).

Upon completion of this analysis, we are able to determine the following major results:

- The morning period between 6AM to 12PM had the most overall crime being reported
- Approximately 35% of all crime reported would be considered violent
- Brookeville has the largest amount of crime being reported for not only violent and non-violent crimes, but also just violent crimes
- The Morning period between 6AM to 12PM has the most violent crime being reported

Please read through this document in detail as the analysis provides a very thorough review of the dataset and breaks each of the tasks mentioned above into many different components. 

## Data Exploration

We are now going to complete a quick data check to get a better understanding of the dataset and what we will be working with. Let's begin by examining the first 10 rows and the shape of the data (number of rows and number of columns).

In [ ]:
import pandas as pd
pd.options.display.max_columns = 25

crime_data = pd.read_csv("MontgomeryCountyCrime2013.csv")

crime_data.head()

In [ ]:
crime_data.info()

The only columns from the above dataset we will be using are the 'Dispatch Date / Time', 'Class Description', 'City', and 'Start Date / Time'. Each of these columns have zero null objects, so hardly any data cleaning needs to be done. When looking at each of these columns we can see that the only one we may need to modify is the time columns as they don't seem to be in a form we can use just yet. 

Since we are not going to be using any columns that have null objects, we can ignore them.

## **Data Analysis**

### Number of Crimes by Month

In [ ]:
#Convert the date to month, represented by a numerical value

crime_data.loc[:,'Dispatch Date / Time'] = pd.to_datetime(crime_data['Dispatch Date / Time'])
month_of_crime = crime_data['Dispatch Date / Time'].dt.month

month_of_crime_sorted = month_of_crime.value_counts().sort_index()
print("The month's of crime committed: \n\n", month_of_crime_sorted)

Using the above value count, this data only contains values from July - December. That is an obvious limitation of the data and we will not be able to make full assumptions for any yearly trends. However, this data will be useful in taking a quick glance at any patterns that arise.

### Proportion of Crimes Over a 24 Hour Period

In [ ]:
# Print out the number of crimes that occur in the day and night 
# day is defined as the time between 6am - 6pm.
hour_crime = crime_data['Dispatch Date / Time'].dt.hour

day_crime = (hour_crime >= 6) & (hour_crime < 18)
day_crime_counts = day_crime.value_counts()

# Re-label the numerical index to represent the actual descriptors
day_crime_counts.index = ['Day', 'Night']

print("The time of crime committed during the day vs night: \n",day_crime_counts)


Let's now compute the percentage of crimes that happen during the day using this formula: 

percent = (number of crimes happening at day / total number of crimes) * 100

In [ ]:
day_night_per = (day_crime_counts['Day'] / (day_crime_counts['Day'] + day_crime_counts['Night'])) * 100
print(day_night_per)

Here, we have found that crimes committed during the day vs night are 59%.

Now let's look at this more in depth, and break these values into the morning, afternoon, evening, and night time intervals.

In [ ]:
# create an empty dictionary for our intervals
crime_by_6hours = {'Night':'','Morning':'',"Afternoon":'', "Evening":''}

i = 0

# create a for loop that goes through the various dictionary keys applying the filter we want
# to for each 6 hour time interval
for item in crime_by_6hours:
    crime_by_6hours[item] = hour_crime[(hour_crime >= i) & (hour_crime < i+6)].shape[0]
    i += 6

# print out the results
print("Crimes broken down by 6 hour intervals: \n", crime_by_6hours)

The in-depth data gives a much better resolution:

 - morning -> 6am - 12pm -> Total crimes reported = 6971
 - afternoon = 12pm - 6pm -> Total crimes reported = 6842
 - evening = 6pm - 12am -> Total crimes reported = 6287
 - night = 12am - 6am -> Total crimes reported = 3269

Now we have a better resolution of the time when crimes are being reported but we don't have a very good understanding of what they mean. Let's try creating a chart that explains this visually, so we can explore it further.

In [ ]:
# This can be visually represented like so:
import matplotlib.pyplot as plt

colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue']

# Makes the chart "pop" out
explode = (0, 0.1, 0, 0)
plt.pie([val for val in crime_by_6hours.values()],explode=explode, labels=[k for k in crime_by_6hours.keys()], 
        colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')
plt.title('Breakdown of 6 Hour Intervals for Crime')
plt.show()


Using this chart, we can easily identify the largest 6-hour time interval being the morning with 29.8%. We can also see that the evening and afternoon intervals are quite similar when compared to the morning interval. Alternatively, we can also see that the night time interval has roughly half the amount of calls being dispatched when compared to the other intervals.

Now, let's look at which day of of the week has the most crimes being reported. To give a better understanding of the data, we will turn these values into percentages.

### Proportion of Crimes by Week days

In order to take percentages for a given weekday we can use this formula:

Probability(a crime is commited on a [given day of the week]) = number of crimes commited on a [given day of the week] / total number of crimes

In [ ]:
weekday_of_crime = crime_data['Dispatch Date / Time'].dt.weekday_name
weekday_of_crime_sorted = weekday_of_crime.value_counts(normalize = True) * 100

print("The days of crime committed on: \n", weekday_of_crime_sorted)

We can see from the above list that Tuesday is the most likely day for a crime to be reported. Now, let's show this data visually by outputting a graph of the above results.

In [ ]:
weekday_of_crime_sorted = weekday_of_crime_sorted.reindex(
     ['Monday','Tuesday','Wednesday', 'Thursday','Friday', 'Saturday','Sunday'])

ax = weekday_of_crime_sorted.plot(kind = 'bar')
plt.style.use('fivethirtyeight')
plt.grid(False)

plt.title('Proportion of crimes by weekday', weight = 'bold', fontsize = 15,color = 'black')

plt.xlabel("Days of the Week", color='black')
plt.xticks(rotation=45,color='black')

plt.yticks(color='black')

vals = ax.get_yticks()
ax.set_yticklabels(['{:3.1f}%'.format(x) for x in vals])
plt.ylabel('Percentage of Crimes Reported', color='black')
plt.show()


This graph allows us to determine that Tuesday is the day of the week that has the largest amount of crimes being reported. 

We can also note that the two days with the least amount of crime reported is Saturday and Sunday.

### Median Dispatch Time of Officers

One last item we should explore before moving on, is using the data to determine the average time it takes police to dispatched to the crime. In order to do this, we can use the *'Dispatch Time'* column and the *'Start Time'* column by subtracting the two to get the difference.

In [ ]:
# The difference in dispatch time and start time can be determined as follows

dispatch_time = pd.to_datetime(crime_data['Dispatch Date / Time'])
start_time = pd.to_datetime(crime_data['Start Date / Time'])

crime_data['Time_Difference'] = (dispatch_time - start_time)
print(crime_data['Time_Difference'].head(15))

Looking at the above output, we can see some of the time differences are quite out of place and are likely an administrative error so we don't want to include these in the calculation. We will need to exclude any value that is over one day from this average as these are outliers and likely errors. 

In order to get a representative value that doesn't take into account any other large outliers that show up throughout the series, it would be best to take the median in this dataset instead of the average. Let's calculate that now.

In [ ]:
# Filter out any values greater than 1 day

crime_data['Time_Difference_days'] = crime_data['Time_Difference'].astype('timedelta64[D]')
day_filter = ~(crime_data['Time_Difference_days'] > 0) & ~(crime_data['Time_Difference_days'] < 0)

avg_response_time = crime_data[day_filter]['Time_Difference'].median()
print(avg_response_time)

After removing the time differences that were greater than 1 day the median time between the start and dispatch time was calculated to be 48 seconds. 

This may seem odd at first however, an important thing to note is that the time difference we calculated is actually just the time of the officers being dispatched to the crime. Not the response time of the police to show up to a 911 call. When we look at the data this way, this median value seems more accurate. Using online materials, an average time to dispatch cannot be found and so we have no way to corroborate this value as to its accuracy.

### Proportion of Crimes by City

Now that we have reviewed and analyzed the time of police dispatches, we can look at which locations crime is being reported at. We first need to choose the column that best represents the areas we want to group the crimes by. The *'City'* column seems like the best column to use. We will now analyze crimes that occur in each city within Montgomery County.

In [ ]:
crime_occurrences_by_city = crime_data['City'].value_counts(normalize=True) * 100
ax = crime_occurrences_by_city.plot(kind='bar', x='City',figsize=(15,5))

plt.style.use('fivethirtyeight')
plt.grid(False)

plt.title("Proportion of Crimes Reported by City",color='black')

plt.xlabel("Cities", color='black')
plt.xticks(color='black')

vals = ax.get_yticks()
ax.set_yticklabels(['{:3.1f}%'.format(x) for x in vals])
plt.ylabel('Percent of Crimes Reported', color='black')
plt.yticks(color='black')

plt.show()

Here we can see that Silver Spring has the most crime being reported. To get a better understanding of this data we should look at the crime per capita (1000 People) of each area. Using the crime per capita will give us a more accurate understanding of crime in relation to the population of each city. Researching online did not yield a dataset to create the per capita value. In order to create the per capita value, each city has been reasearched and a dictionary has created to calculate the value we are looking for.

In [ ]:
pop_by_city ={'SILVER SPRING': 71452,'ROCKVILLE': 61209,'GAITHERSBURG': 66816, 'GERMANTOWN': 90676, 
              'BETHESDA': 60858,'MONTGOMERY VILLAGE': 32032, 'POTOMAC': 44965, 'CHEVY CHASE': 2959, 
              'OLNEY': 33844,'KENSINGTON': 2313,'BURTONSVILLE':13344,'DERWOOD':15600,'DAMASCUS':11430,
              'CLARKSBURG':13677,'TAKOMA PARK':17670,'POOLESVILLE':5160,'BOYDS':10460,'BROOKEVILLE':134,
              'SANDY SPRING':5648,'DICKERSON':1848,'ASHTON':5648,'SPENCERVILLE':15594,
              'WASHINGTON GROVE':573,'BRINKLOW':154,'BARNESVILLE':178,'GLEN ECHO':269,'MOUNT AIRY':9388,
              'LAUREL':26160,'BEALLSVILLE':111,'HYATTSVILLE':18420} 

crime_per_capita = {}


crime_per_capita = {city:((crime_occurrences_by_city.loc[city]/pop)*1000) for (city, pop) in pop_by_city.items()}


crimes_per_1000_people = pd.DataFrame({'City': list(crime_per_capita.keys()), 
              'Crimes per 1000 people': list(crime_per_capita.values())})
crimes_per_1000_people = crimes_per_1000_people.set_index('City')
crimes_per_1000_people.head(10)



Let's turn the above data into a graph so we can analyze the data visually.

In [ ]:
crimes_per_1000_people.sort_values('Crimes per 1000 people',ascending=False, inplace=True)
crimes_per_1000_people.plot(kind='bar',figsize=(15,5))

plt.style.use('fivethirtyeight')
plt.grid(False)

plt.title("Crimes per Capita Reported by City",color='black')

plt.xlabel("Cities", color='black')
plt.xticks(color='black')


vals = ax.get_yticks()
ax.set_yticklabels(['{:3.1f}%'.format(x) for x in vals])
plt.ylabel('Crimes Reported per Capita', color='black')
plt.yticks(color='black')

plt.gca().legend_.remove()
plt.show()

Now that we can see the crime per capita, we have a better understanding of the data itself. We would have expected Silver Spring to lead the pack as it had over 2 times the amount of crimes being reported than the next closest city.  This is why it is important that a per capita value is used when comparing large population centres to smaller ones.

In this graph, Brookeville has the largest amount of crime that is being reported and has a population of 134 people. Chevy Chase and Kensington towns are the next highest and have 2959 and 2313 people located within them, respectively. 

### Analyzing Violent Crimes

Because this value includes all crimes, we cannot yet determine how safe these cities and towns are. We first need to look at what types of crimes are being committed and whether or not they are violent. Then we can comment on which locations should be avoided.

In [ ]:
# How often do specific crimes occur and what is the count of each type of reported crime

types_of_crime_count = crime_data['Class Description'].value_counts()
pd.options.display.max_rows = 10000
types_of_crime_count.head(1000)


We can now use the above list to help choose which specific description is non-violent or violent. We can certainly see that anything with "Assault" or "Asslt" in the name is a violent crime, but there are also a few other keywords such as "Rape","Homicide", "Knife","Firearm","Armed","Arson", "Robbery", and "Larceny" that also fit our violent crime definition.

Using these keywords, we will now determine the percentage of violent vs non-violent crime.

In [ ]:
# Now let's try to identify which of these are violent and which are non-violent

# Use Regex to determine which are violent and which are non-violent
crime_data['Violent crime'] = crime_data['Class Description'].str.contains(
    'ASSAULT|ASSLT|RAPE|HOMICIDE|KNIFE|FIREARM|ARMED|ARSON|ROBBERY|LARCENY')

crime_data.loc[crime_data['Violent crime']==True,'Violent crime'] = "Violent"
crime_data.loc[crime_data['Violent crime']==False,'Violent crime'] = "Non-Violent"
crime_data_value_counts = crime_data['Violent crime'].value_counts(normalize = True) * 100
crime_data_value_counts.head()

After identifying which fields in the dataframe are "Violent" and which are "Non-Violent" we can see that 34.9% of all crimes reported are actually violent.

### Analyzing Violent Crimes by City

Now we need to determine where these violent crimes are occurring.

In [ ]:
violent_crime = crime_data[crime_data['Violent crime']=="Violent"]

print(violent_crime['City'].value_counts(normalize=True)*100)

Looking at the percentage of violent crimes reported we can again see that Silver Spring has the most. Let's look at this in a visual form as well.

In [ ]:
violent_crime_counts = violent_crime['City'].value_counts(normalize=True)*100

ax = violent_crime_counts.plot(kind='bar', x='City',figsize=(10,5))

plt.style.use('fivethirtyeight')
plt.grid(False)

plt.title("Number of Violent Crimes Reported by City")

plt.xlabel("Cities")


plt.ylabel('Violent Crimes Reported')
vals = ax.get_yticks()
ax.set_yticklabels(['{:3.1f}%'.format(x) for x in vals])

plt.show()

Knowing how drastically our results changed last time, we should also use our population per capita calculation and graphically determine which city has the largest amount of violent crimes being reported per capita.

In [ ]:
pop_by_city_2 = {'SILVER SPRING': 71452,'ROCKVILLE': 61209,'GAITHERSBURG': 66816, 'GERMANTOWN': 90676, 
              'BETHESDA': 60858,'MONTGOMERY VILLAGE': 32032, 'POTOMAC': 44965, 'CHEVY CHASE': 2959, 
              'OLNEY': 33844,'KENSINGTON': 2313,'BURTONSVILLE':13344,'DERWOOD':15600,'DAMASCUS':11430,
              'CLARKSBURG':13677,'TAKOMA PARK':17670,'POOLESVILLE':5160,'BOYDS':10460,'BROOKEVILLE':134,
              'SANDY SPRING':5648,'DICKERSON':1848,'ASHTON':5648} 

violent_crime_per_capita = {}

violent_crime_per_capita = {city: (violent_crime_counts.loc[city]/pop)*1000 for (city, pop) in pop_by_city_2.items()}

violent_crimes_per_1000_people = pd.DataFrame({'City': list(violent_crime_per_capita.keys()), 
              'Violent Crimes per 1000 people': list(violent_crime_per_capita.values())})
violent_crimes_per_1000_people = violent_crimes_per_1000_people.set_index('City')
violent_crimes_per_1000_people.head(30)

Now that we have our per capita values, we are going to plot them in a bar graph just as we did before, analyzing the data visually.

In [ ]:
violent_crimes_per_1000_people.sort_values('Violent Crimes per 1000 people',ascending=False, inplace=True)
violent_crimes_per_1000_people.plot(kind='bar',figsize=(15,5))

plt.style.use('fivethirtyeight')
plt.grid(False)

plt.title("Violent Crimes per Capita Reported by City",color='black')

plt.xlabel("Cities", color='black')
plt.xticks(color='black')

plt.ylabel('Violent Crimes Reported per Capita', color='black')
plt.yticks(color='black')

plt.gca().legend_.remove()
plt.show()

Looking at the above graph, we can see that 'Brookeville' has the largest amount of violent crimes being reported per capita in the county. If we remember back to the 'Crimes per Capita Reported by City' graph, Brookeville was also the largest. This was not expected because the population of Brookeville was so small, with a population of 134, but it ended up having the most crime being reported per capita.

Now that we know where the most violent and non-violent crimes are being reported we should maybe avoid Brookeville alltogether. 

### Proportion of Violent Crimes Over a 24 Hour Period

Using the Violent crime information, let's also try to find out when we should be avoiding this area. The best way to do this would to be to use our day/night method and break it into the 6 hour intervals for further analysis.

In [ ]:
hour_violent_crime = violent_crime['Dispatch Date / Time'].dt.hour
day_violent_crime = (hour_violent_crime >= 6) & (hour_violent_crime < 18)
day_violent_crime_counts = day_violent_crime.value_counts()
day_violent_crime_counts.index = ['Day', 'Night']

print("The time of crime committed during the day vs night: \n",day_violent_crime_counts)

It seems like most violent crimes are reported during the day. Now, let's again try to find a specific 6 hour window that these violent crimes are being reported.

In [ ]:
# create an empty dictionary for our intervals
violent_crime_by_6hours = {'Night':'','Morning':'',"Afternoon":'', "Evening":''}
i = 0

# create a for loop that goes through the various dictionary keys applying the filter we want
# to for each 6 hour time interval
for item in violent_crime_by_6hours:
    violent_crime_by_6hours[item] = hour_violent_crime[(hour_crime >= i) & (hour_violent_crime < i+6)].shape[0]
    i += 6

# print out the results
print("Crimes broken down by 6 hour intervals: \n", violent_crime_by_6hours)

Using the same chart as before, let's compare the results.

In [ ]:
colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue']
explode = (0, 0.1, 0, 0)
plt.pie([val for val in violent_crime_by_6hours.values()],explode=explode, labels=[k for k in violent_crime_by_6hours.keys()], colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')
plt.title('Breakdown of 6 Hour Intervals for Violent Crime')
plt.show()

Before we analyze this chart, we should remember the chart for both all crimes being reported. When looking at the previous chart, the morning and afternoon intervals contained 29.8% and 29.3% of all crimes being reported, respectively.

In this chart we see that the morning interval was the largest with 36.8%, roughly a 7% increase. Whereas the afternoon interval made up 34.9% of all violent crimes being reported, this is roughly a 6% increase.

Without a doubt, we can say that the morning and afternoon intervals are the most dangerous time to be out and about in any given town in Montgomery County.

## Conclusion

Using the dataset from Montgomery County we were able to analyze and determine several key points: 

- October was the month that had the most crime being recorded. However, because the dataset only has info from July to October we cannot conclusively determine that this is the largest month throughout the year. Therefore, further yearly trends cannot be determined from this dataset.

- Over a 24 hour period, we determined that the day is when most crimes are being reported. Furthermore, we wanted more resolution on this time frame and split out the 24 hour period into four six-hour intervals. Using this data, it was determined that the morning was the largest with the afternoon a close second 29.8% and 29.3%, respectively. Both of these values were roughly double the amount of the night time value. The reason the night time value may be so much lower is that there are less police officers patrolling the area and able to be dispatched.

- The day of the week that had the most crime being reported was Tuesday. The weekend, Saturday and Sunday, had the least amount of crime being reported. This again could fit our theory that there are less police officers available to be dispatched and so those days are going have less crime reported.

- The median dispatch time for officers was 48 seconds. However, because we were not able to compare our results using any other data found through online research, we cannot conclusively say whether this a is good or bad result. As this is not the actual response time for a crime it may not be as important, but should still be considered an important aspect of responding to a crime.

- Brookeville was found to have the largest per capita amount of crimes being recorded with approximately of 2 crimes reported per 1000 people (per capita). This is interesting because this town only contains a population of 134 people. The largest amount of crime reported by a city was Silver Spring with roughly 35% of all crime occurring there but only had a per capita value of approximately 0.5, which was the fourth largest.

- The amount of violent crime that were committed in Montgomery county was 35%, while the non-violent crime was 65%. In order to determine which crimes were violent and which were not, a unique list of each crime description was printed out. Each unique value was then looked through to confirm whether or not it was violent.

- The number of violent crimes per capita was found again to be Brookeville with a per capita value of roughly 1.2 per capita. Silver Spring was again determined to have the largest amount of violent crime reported. This should be no surprise as the location with the most crime should also have the most violent crime occurring there as well.

- For violent crimes, we again determined that the day is when most violent crimes were reported. Furthermore, breaking this into a 6-hour interval we found that the morning and afternoon  had the largest amount of violent crime being reported with 36.8% and 34.9%, respectively. In these time intervals, we can note an increase in both morning and afternoon by 7% and 6%, respectively, when compared to our previous analysis of total crimes being reported.

**In Conclusion**

Without more information on a country wide crime rate, it may be hard to conclusively determine which cities or towns should be avoided, after looking at all the information. Using [wikipedia](https://en.wikipedia.org/wiki/List_of_United_States_cities_by_crime_rate) we can actually determine a Violent Crime Rate per Capita value for Baltimore, the capital of Maryland, and use this as a benchmark to compare our values.

After completing this calculation, we have found that Baltimore, Maryland has a violent crime rate of 2.47 per Capita. This means that Brookville is still twice as safe as Baltimore. Also, we can say that any town within Montgomery County is safer than the Capital of the state itself.

However, you should always use caution when visiting these cities in Montgomery County, especially between the Morning and Afternoon time intervals as that seems to be when the majority of crime is reported.